# Smart Factory Energy Prediction Challenge

## Problem Overview
The goal is to develop a machine learning model to accurately predict the energy consumption of industrial equipment (`equipment_energy_consumption`) based on sensor data from a manufacturing facility. This will help optimize operations for energy efficiency and cost reduction.

## Objectives
1. Analyze sensor data to identify patterns and relationships.
2. Build a robust regression model.
3. Evaluate model performance using RMSE, MAE, and R².
4. Provide actionable insights and recommendations.
5. Determine the utility of `random_variable1` and `random_variable2`.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Scikit-learn imports
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Format floats to 3 decimal places
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Load and Inspect Data

In [2]:
# Load the dataset
try:
    df = pd.read_csv('data/data.csv')
except FileNotFoundError:
    print("Error: 'data/data.csvdata.csv' not found. Please ensure the data file is in the correct path.")
    
    exit()


print("Dataset Shape:", df.shape)
print("\nFirst 5 rows of the dataset:")
display(df.head())

print("\nDataset Info:")
df.info()

print("\nSummary Statistics:")
display(df.describe(include='all'))

Dataset Shape: (16857, 29)

First 5 rows of the dataset:


,timestamp,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,zone5_temperature,zone5_humidity,zone6_temperature,zone6_humidity,zone7_temperature,zone7_humidity,zone8_temperature,zone8_humidity,zone9_temperature,zone9_humidity,outdoor_temperature,atmospheric_pressure,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2
0,2016-01-11 17:00:00,60.0,-77.78778596503064,33.74660933896648,47.59666666666671,19.2,44.790,19.790,NaN,19.000,45.567,17.167,55.200,NaN,84.257,17.200,41.627,18.200,48.900,17.033,45.530,6.600,733.500,92.000,7.000,63.000,5.300,13.275,13.275
1,2016-01-11 17:10:00,60.0,30.0,19.89,46.69333333333329,19.2,44.722,19.790,44.790,19.000,45.992,17.167,55.200,6.833,84.063,17.200,NaN,18.200,48.863,17.067,45.560,6.483,733.600,92.000,6.667,59.167,5.200,18.606,18.606
2,2016-01-11 17:20:00,50.0,30.0,19.89,46.3,19.2,44.627,19.790,44.933,35.921,45.890,NaN,55.090,6.560,83.157,17.200,41.433,18.200,48.730,17.000,45.500,6.367,733.700,92.000,6.333,55.333,5.100,28.643,28.643
3,2016-01-11 17:30:00,50.0,40.0,33.74660933896648,46.0666666666667,19.2,44.590,19.790,45.000,NaN,45.723,17.167,55.090,6.433,83.423,17.133,41.290,18.100,94.386,17.000,45.400,6.250,733.800,92.000,6.000,51.500,37.674,45.410,45.410
4,2016-01-11 17:40:00,60.0,40.0,19.89,46.33333333333329,19.2,44.530,19.790,45.000,18.890,45.530,17.200,55.090,6.367,84.893,17.200,41.230,18.100,48.590,4.477,45.400,6.133,733.900,92.000,5.667,47.667,4.900,10.084,10.084



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16857 entries, 0 to 16856
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   timestamp                     16857 non-null  object 
 1   equipment_energy_consumption  16013 non-null  object 
 2   lighting_energy               16048 non-null  object 
 3   zone1_temperature             15990 non-null  object 
 4   zone1_humidity                16056 non-null  object 
 5   zone2_temperature             16004 non-null  object 
 6   zone2_humidity                15990 non-null  float64
 7   zone3_temperature             16055 non-null  float64
 8   zone3_humidity                15979 non-null  float64
 9   zone4_temperature             16041 non-null  float64
 10  zone4_humidity                16076 non-null  float64
 11  zone5_temperature             16019 non-null  float64
 12  zone5_humidity                16056 non-null 

,timestamp,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,zone5_temperature,zone5_humidity,zone6_temperature,zone6_humidity,zone7_temperature,zone7_humidity,zone8_temperature,zone8_humidity,zone9_temperature,zone9_humidity,outdoor_temperature,atmospheric_pressure,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2
count,16857,16013,16048,15990,16056,16004,15990.000,16055.000,15979.000,16041.000,16076.000,16019.000,16056.000,16009.000,16010.000,16063.000,16052.000,16009.000,16080.000,16084.000,15969.000,16051.000,16015.000,16058.000,16029.000,16042.000,16031.000,16031.000,16033.000
unique,16769,130,20,433,2172,838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2016-01-25 21:50:00,50.0,0.0,21.0,3.348059697903068,19.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,3400,11687,479,164,303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,39.495,21.666,38.201,20.240,37.946,19.053,50.289,6.470,59.163,19.672,34.033,21.606,41.854,18.851,40.318,6.219,755.758,78.978,4.196,38.457,2.784,24.855,25.094
std,NaN,NaN,NaN,NaN,NaN,NaN,10.130,2.594,10.144,2.783,10.770,2.346,18.723,8.868,52.658,2.880,11.345,2.975,12.302,2.529,11.169,7.555,13.644,28.566,4.410,21.319,6.095,26.215,25.524
min,NaN,NaN,NaN,NaN,NaN,NaN,-77.266,6.544,-71.406,4.613,-81.446,5.921,-141.640,-42.987,-353.393,3.578,-84.883,4.502,-94.386,4.477,-81.582,-37.525,678.160,-221.669,-20.930,-82.330,-32.098,-120.170,-120.410
25%,NaN,NaN,NaN,NaN,NaN,NaN,37.758,20.533,36.593,19.267,35.200,18.061,45.290,2.930,37.067,18.500,31.000,20.500,38.627,17.890,38.230,3.000,750.800,71.000,2.000,29.000,0.450,12.180,12.194
50%,NaN,NaN,NaN,NaN,NaN,NaN,40.293,21.767,38.400,20.290,38.090,19.050,48.854,6.263,62.767,19.600,34.230,21.790,42.040,18.890,40.363,6.000,756.200,84.167,4.000,40.000,2.750,24.867,24.834
75%,NaN,NaN,NaN,NaN,NaN,NaN,43.000,22.760,41.433,21.357,41.561,20.100,53.918,9.690,86.590,21.000,38.157,22.790,46.004,20.200,43.790,9.250,762.100,91.988,6.000,40.000,5.308,37.950,37.972


### Initial Observations:
- The dataset has 16857 rows and 29 columns.
- `timestamp` is an object, needs conversion to datetime.
- Several columns, including the target `equipment_energy_consumption` and other sensor readings like `lighting_energy`, `zone1_temperature`, etc., are read as `object` type. These likely contain numerical data mixed with non-numeric strings and need to be converted to numeric.
- Many columns have missing values.
- Some features show a very wide range (e.g., min/max values for humidity, temperature in some zones), suggesting potential outliers or data entry errors.